# 4. Decision Tree Model

Decision Tree를 이용한 가벼운 회귀 모델을 구현합니다.

## 개요
- **모델**: Decision Tree Regressor
- **데이터**: Melbourne Housing Snapshot
- **목표**: 주택 가격 예측

## 1. 데이터 로드 및 전처리

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 데이터 로드
melbourne_file_path = './input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# 결측치 제거
melbourne_data = melbourne_data.dropna(axis=0)

print(f"데이터 크기: {melbourne_data.shape}")
print(f"\n데이터 컬럼:")
print(melbourne_data.columns.tolist())

## 2. Feature 선택

주택 가격 예측을 위한 주요 Feature들을 선택합니다.

In [ ]:
# 예측 대상 (target)
y = melbourne_data.Price

# Feature 선택 (수치형 변수들)
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                      'YearBuilt', 'Lattitude', 'Longtitude']

X = melbourne_data[melbourne_features]

print("Features:")
print(X.describe())

## 3. Train/Test Split

데이터를 학습용과 검증용으로 분리합니다.

In [ ]:
# 학습/테스트 데이터 분리 (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

## 4. Decision Tree 모델 정의 및 학습

### Decision Tree란?
- 데이터를 분할하여 트리 구조로 예측하는 알고리즘
- 해석이 용이하고 구현이 간단함
- 과적합(Overfitting)에 주의 필요

### 주요 하이퍼파라미터
- `max_depth`: 트리의 최대 깊이 (과적합 방지)
- `min_samples_split`: 노드를 분할하기 위한 최소 샘플 수
- `min_samples_leaf`: 리프 노드의 최소 샘플 수
- `random_state`: 재현성을 위한 시드

In [ ]:
# Decision Tree 모델 생성 (가벼운 모델을 위해 max_depth 제한)
model = DecisionTreeRegressor(
    max_depth=5,           # 트리 깊이 제한 (가벼운 모델)
    min_samples_split=20,  # 분할을 위한 최소 샘플 수
    min_samples_leaf=10,   # 리프 노드 최소 샘플 수
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

print("모델 학습 완료!")
print(f"트리 깊이: {model.get_depth()}")
print(f"리프 노드 수: {model.get_n_leaves()}")

## 5. 예측 및 평가

In [ ]:
# 예측
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 평가 지표 계산
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("=" * 50)
print("모델 평가 결과")
print("=" * 50)
print(f"\n[학습 데이터]")
print(f"  MAE (Mean Absolute Error): ${train_mae:,.2f}")
print(f"  RMSE (Root Mean Squared Error): ${train_rmse:,.2f}")
print(f"  R² Score: {train_r2:.4f}")

print(f"\n[테스트 데이터]")
print(f"  MAE (Mean Absolute Error): ${test_mae:,.2f}")
print(f"  RMSE (Root Mean Squared Error): ${test_rmse:,.2f}")
print(f"  R² Score: {test_r2:.4f}")
print("\n" + "=" * 50)

## 6. Feature Importance

어떤 Feature가 가격 예측에 가장 중요한지 확인합니다.

In [ ]:
# Feature 중요도
feature_importance = pd.DataFrame({
    'feature': melbourne_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature 중요도:")
print(feature_importance.to_string(index=False))

## 7. 예측 예시

In [ ]:
# 처음 5개 샘플에 대한 예측
sample_predictions = pd.DataFrame({
    'Actual Price': y_test.iloc[:5].values,
    'Predicted Price': y_test_pred[:5],
    'Difference': y_test.iloc[:5].values - y_test_pred[:5]
})

print("\n실제 가격 vs 예측 가격 (처음 5개 샘플):")
print(sample_predictions.to_string(index=False))

## 8. 결론

### Decision Tree 모델의 장점
- 해석이 용이함 (Feature Importance 확인 가능)
- 빠른 학습 및 예측 속도
- 데이터 스케일링 불필요
- 비선형 관계 포착 가능

### Decision Tree 모델의 단점
- 과적합 경향 (max_depth, min_samples_split 등으로 조절 필요)
- 데이터 변화에 민감함
- 예측 성능이 앙상블 모델에 비해 낮을 수 있음

### 개선 방향
- Random Forest, Gradient Boosting 등 앙상블 모델 사용
- 하이퍼파라미터 튜닝 (GridSearchCV, RandomizedSearchCV)
- 추가 Feature Engineering
- Cross-Validation을 통한 모델 안정성 확보